<a href="https://colab.research.google.com/github/fmbento/Jupyter-Notebooks_ad-hoc-solutions/blob/main/Teste_com_vers%C3%A3o_OSS_gratuita_do_Whisper_da_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseado na ideia original  partilhada neste post:

`https://deepgram.com/learn/how-to-run-openai-whisper-in-google-colab`

<br/>

### 1. Instalar o *Whisper* do repo oficial:

In [ ]:
!pip install git+https://github.com/openai/whisper.git

Video do Youtube a transcrever:

In [ ]:
!pip install yt-dlp
!yt-dlp https://youtu.be/GjpgfHqy7IM?si=opwTO9MFiph5MvAn --format m4a -o "%(id)s.%(ext)s"

Correr o Whisper

In [5]:
!whisper "/content/GjpgfHqy7IM.m4a" --model medium --language Portuguese

100%|█████████████████████████████████████| 1.42G/1.42G [00:36<00:00, 41.6MiB/s]
[00:00.000 --> 00:13.040]  Nos últimos dias, sucedem-se notícias sobre as previsões dos resultados do Mundial de
[00:13.040 --> 00:15.760]  Futebol, que está mesmo mesmo a começar.
[00:15.760 --> 00:21.400]  Falta cá o famoso povo Pôl para nos dizer de forma certeira qual é a equipa que vai
[00:21.400 --> 00:22.400]  vencer.
[00:22.400 --> 00:29.680]  Ainda há dias, o DN dava a notícia que uma empresa tinha feito os seus cálculos e apontava
[00:29.680 --> 00:34.480]  a equipa russa como vencedora.
[00:34.480 --> 00:41.840]  Passados uns dias na TSF, ouvia-se ainda que a empresa de videojogos, a Electronic Arts,
[00:41.840 --> 00:46.800]  também tinha feito os seus cálculos e dava a vitória à equipa francesa.
[00:46.800 --> 00:49.800]  Poderemos confiar nestas previsões?
[00:49.800 --> 00:50.800]  Quem irá ganhar?
[00:50.800 --> 00:53.120]  A estatística pode ajudar.
[00:53.120 --> 00:58.080]  Com efeito, m

### Verificar a qualidade

In [7]:
import pandas as pd
import numpy as np
import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

 `webvtt-py` lê o formato de legenda *webvtt*:

In [ ]:
!pip install webvtt-py
import webvtt

`librosa` é uma biblioteca para ler e manipular ficheiros de aúdio.

In [10]:
import librosa

Precisamos converter H:M:S em segundos e cortar pedaçõs do áudio (entre `start` e `end`)

In [11]:
def simple_hms(s):
  h,m,sec = [float(x) for x in s.split(':')]
  return 3600*h + 60*m + sec

def trim_audio(row,audio,sample_rate):
  t = np.arange(len(audio))
  t = t/sample_rate
  f = np.where( (t>=row.start_s) & (t<=row.end_s) )
  return audio[f]

Vamos usar então a `webvtt` to para ler a transcrição e a `librosa` para ouvir o aúdio.

In [15]:
transcript = webvtt.read('/content/GjpgfHqy7IM.vtt')
audio,sample_rate = librosa.load('/content/GjpgfHqy7IM.m4a')

Vamos colocar isto num dataframe panda para depois podermos escolher uma amostra, aleatoriamente:

In [16]:
df = pd.DataFrame(columns=['start','end','text'])

df['start'] = [x.start for x in transcript]
df['end'] = [x.end for x in transcript]
df['text'] = [x.text for x in transcript]
df['start_s'] = df['start'].apply(simple_hms)
df['end_s'] = df['end'].apply(simple_hms)
df['audio'] = df.apply(trim_audio,axis=1,args=(audio,sample_rate))
df.head()

,start,end,text,start_s,end_s,audio
0,00:00:00.000,00:00:13.040,"Nos últimos dias, sucedem-se notícias sobre as...",0.00,13.04,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,00:00:13.040,00:00:15.760,"Futebol, que está mesmo mesmo a começar.",13.04,15.76,"[0.032274634, 0.034167636, 0.054932613, 0.0612..."
2,00:00:15.760,00:00:21.400,Falta cá o famoso povo Pôl para nos dizer de f...,15.76,21.40,"[-0.003696789, -0.0044890745, -0.013991337, -0..."
3,00:00:21.400,00:00:22.400,vencer.,21.40,22.40,"[0.045765854, 0.05497848, 0.065051235, 0.07472..."
4,00:00:22.400,00:00:29.680,"Ainda há dias, o DN dava a notícia que uma emp...",22.40,29.68,"[0.040475093, 0.037200846, 0.025117809, 0.0206..."


... e a amostragem e comparação (correr várias vezes para obter amostras diferentes):

In [20]:
segment = df.sample(n=1).iloc[0]
# segment = df.loc[16]
print(segment.text)
ipd.Audio(segment.audio,rate=sample_rate)

Nos anos 80, na Estatística Neerlândica, foi proposto um modelo de Poisson para precisamente
